In [1]:
import org.apache.spark.sql._
import org.apache.spark.ml._
import java.io._
import play.api.libs.json._
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._

In [2]:
val vehicleUsers = spark.read.json("/labs/laba02/autousers.json").toDF()

Waiting for a Spark session to start...

vehicleUsers = [autousers: array<string>]


[autousers: array<string>]

In [9]:
val logs = spark.read.option("delimiter", "\t").textFile("/labs/laba02/logs")

logs = [value: string]


[value: string]

In [10]:
logs.show(3, truncate=false)

+-----------------------------------------------------------------------------------------------------+
|value                                                                                                |
+-----------------------------------------------------------------------------------------------------+
|258131083251	1423724291.637	http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251	1423724290.637                                                                          |
|182984926014	1423724291.666	http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
+-----------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [11]:
val col_ = split(col("value"), "\t").alias("value")
val logs_splited = logs.select(col_)
logs_splited.show(3, truncate=false)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|[258131083251, 1423724291.637, http%3A%2F%2Fwww.zakon.kz%2F]                                             |
|[258131083251, 1423724290.637]                                                                           |
|[182984926014, 1423724291.666, http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5]|
+---------------------------------------------------------------------------------------------------------+
only showing top 3 rows



col_ = split(value, 	) AS `value`
logs_splited = [value: array<string>]


[value: array<string>]

In [12]:
val logs_df = logs_splited.withColumn("UID", $"value"(0))
                    .withColumn("Timestamp", $"value"(1))
                    .withColumn("URL", $"value"(2))
                    .withColumn("Timestamp", to_timestamp(from_unixtime($"Timestamp")))
                    .select("UID","Timestamp", "URL")


logs_df = [UID: string, Timestamp: timestamp ... 1 more field]


[UID: string, Timestamp: timestamp ... 1 more field]

In [28]:
logs_df.withColumn("URL", regexp_replace($"URL", lit("http"), lit(""))).show(3)

+------------+-------------------+--------------------+
|         UID|          Timestamp|                 URL|
+------------+-------------------+--------------------+
|258131083251|2015-02-12 09:58:11|%3A%2F%2Fwww.zako...|
|258131083251|2015-02-12 09:58:10|                null|
|182984926014|2015-02-12 09:58:11|%3A%2F%2Fwww.bmwc...|
+------------+-------------------+--------------------+
only showing top 3 rows



In [ ]:
substring('Spark SQL', 5)

In [69]:
val substrDel = substring_index($"URL", ".", 1)
logs_df.select($"URL",  
               substring(regexp_replace($"URL",substrDel , lit("")), 2, 999).as("c")
              ).show(numRows=5, truncate=200, vertical=true)

-RECORD 0------------------------------------------------------------------------
 URL | http%3A%2F%2Fwww.zakon.kz%2F                                              
 c   | zakon.kz%2F                                                               
-RECORD 1------------------------------------------------------------------------
 URL | null                                                                      
 c   | null                                                                      
-RECORD 2------------------------------------------------------------------------
 URL | http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5 
 c   | bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5                  
-RECORD 3------------------------------------------------------------------------
 URL | http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D4 
 c   | bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D4                  
-RECORD 4-------

substrDel = substring_index(URL, ., 1)


substring_index(URL, ., 1)

In [30]:
logs_df.withColumn("URL", trim(LEADING, lit("http"),  $"URL")).show(3)

Name: Unknown Error
Message: <console>:60: error: not found: value LEADING
       logs_df.withColumn("URL", trim(LEADING, lit("http"),  $"URL")).show(3)
                                      ^

StackTrace: 

In [ ]:
df5.withColumn("sentence_without_label", regexp_replace($"sentence" , lit($"label"), lit("" )))


In [13]:
logs_df.show(10, truncate=false)

+------------+-------------------+-------------------------------------------------------------------------+
|UID         |Timestamp          |URL                                                                      |
+------------+-------------------+-------------------------------------------------------------------------+
|258131083251|2015-02-12 09:58:11|http%3A%2F%2Fwww.zakon.kz%2F                                             |
|258131083251|2015-02-12 09:58:10|null                                                                     |
|182984926014|2015-02-12 09:58:11|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D5|
|182984926014|2015-02-12 09:58:10|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fshowthread.php%3Ft%3D875590%26page%3D4|
|289380960531|2015-02-12 09:58:11|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2Fforumdisplay.php%3Ff%3D128            |
|289380960531|2015-02-12 09:58:10|http%3A%2F%2Fwww.bmwclub.ru%2Fvb%2F                                      |
|85356233460 |2015-

In [14]:
logs_df.printSchema()

root
 |-- UID: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- URL: string (nullable = true)



In [16]:
val vehicleUsers_df = vehicleUsers.select(explode($"autousers").alias("autousers"))
vehicleUsers_df.show(3)

+------------+
|   autousers|
+------------+
|100341861572|
|100473724387|
|100528753939|
+------------+
only showing top 3 rows



vehicleUsers_df = [autousers: string]


[autousers: string]